In [2]:
import numpy as np
import json
import os
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [3]:
def generate_nodes(N, space_size=400, filename="nodes.json", initial_energy=100.0):
    # Sinh N node cảm biến ngẫu nhiên trong không gian 3D với residual energy
    
    # N: số lượng node
    # space_size: kích thước không gian
    # filename: tên file lưu
    # initial_energy: năng lượng ban đầu (E0)

    
    np.random.seed(0)  # để kết quả lặp lại
    node_positions = np.random.rand(N, 3) * space_size
    
    # Sinh residual energy ngẫu nhiên (50-100% của initial energy)
    np.random.seed(42)  # Seed khác để energy không phụ thuộc vào vị trí
    residual_energies = np.random.uniform(0.5 * initial_energy, initial_energy, N)

    data = []
    for i in range(N):
        data.append({
            "id": i,
            "x": float(node_positions[i][0]),
            "y": float(node_positions[i][1]),
            "z": float(node_positions[i][2]),
            # "residual_energy": float(residual_energies[i]),
            "residual_energy": float(initial_energy),
            "initial_energy": float(initial_energy)
        })

    
    os.makedirs("input_data", exist_ok=True)
    filepath = f"input_data/{filename}"
    with open(filepath, "w") as f:
        json.dump(data, f, indent=4)

    print(f"Đã tạo file {filepath} chứa {N} node với residual energy.")
    print(f"Energy range: {residual_energies.min():.1f} - {residual_energies.max():.1f}")

# Sinh dữ liệu với energy information
for N in [50, 100, 150, 200, 250, 300]:
    generate_nodes(N, space_size=100, filename=f"nodes_{N}.json", initial_energy=100.0)

Đã tạo file input_data/nodes_50.json chứa 50 node với residual energy.
Energy range: 51.0 - 98.5
Đã tạo file input_data/nodes_100.json chứa 100 node với residual energy.
Energy range: 50.3 - 99.3
Đã tạo file input_data/nodes_150.json chứa 150 node với residual energy.
Energy range: 50.3 - 99.3
Đã tạo file input_data/nodes_200.json chứa 200 node với residual energy.
Energy range: 50.3 - 99.3
Đã tạo file input_data/nodes_250.json chứa 250 node với residual energy.
Energy range: 50.3 - 99.3
Đã tạo file input_data/nodes_300.json chứa 300 node với residual energy.
Energy range: 50.3 - 99.5


In [ ]:
def calculate_number_clusters(nodes, base_station=(200, 200, 400), space_size=400):
    """
    Tính số cụm optimal theo công thức
    K = √(NL / pi*d_tobs)
    
    nodes: tọa độ 3D của các node
    base_station: tọa độ base station (mặc định tại gốc tọa độ)
    """
    N = len(nodes)
    base_pos = np.array(base_station)
    
    # Tính khoảng cách trung bình từ các node đến base station
    distances = np.linalg.norm(nodes - base_pos, axis=1)
    d_tobs = np.mean(distances)
    
    # Áp dụng công thức K = √(N*L / pi*d_tobs)
    K_optimal = np.sqrt(N * space_size / (np.pi * d_tobs))
    K_optimal = max(1, int(np.round(K_optimal)))  # Đảm bảo K >= 1
    
    print(f"N = {N}, d_tobs = {d_tobs:.2f}, K_optimal = {K_optimal}")
    return K_optimal

In [ ]:
def calculate_objective_function(nodes, labels, centers):
    #  Tính hàm mục tiêu k means
    numerator = 0
    for i in range(2):
        cluster_nodes = nodes[labels == i]
        if len(cluster_nodes) > 0:
            distances = np.linalg.norm(cluster_nodes - centers[i], axis=1)
            numerator += np.sum(distances)
    
    denominator = np.linalg.norm(centers[0] - centers[1])
    
    if denominator == 0:
        return float('inf')
    
    T = numerator / denominator
    return T

def check_subgroup_threshold(nodes, r_sen):
    # Khoảng cách Euclidean tối đa giữa hai node bất kỳ trong cụm <= r_sen

    if len(nodes) <= 1:
        return True
    
    # Tính khoảng cách giữa tất cả các cặp node
    max_distance = 0
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            dist = np.linalg.norm(nodes[i] - nodes[j])
            max_distance = max(max_distance, dist)
    
    return max_distance <= r_sen

def kmeans_with_best_T(nodes, N=30):
    """
    Thực hiện K-means N lần với các tâm khởi tạo ngẫu nhiên khác nhau
    và chọn kết quả có giá trị T nhỏ nhất
    """
    best_T = float('inf')
    best_labels = None
    best_centers = None
    
    for _ in range(N):
        kmeans = KMeans(n_clusters=2, n_init=1)
        labels = kmeans.fit_predict(nodes)
        centers = kmeans.cluster_centers_
        
        T = calculate_objective_function(nodes, labels, centers)
        
        if T < best_T:
            best_T = T
            best_labels = labels.copy()
            best_centers = centers.copy()
    
    return best_labels, best_centers, best_T

def cluster_split(nodes, node_ids, node_data=None, r_sen=100, R=20, N=30, max_depth=10, depth=0):
    """
    K-Means
    
    nodes: tọa độ 3D của các node
    node_ids: list id tương ứng của các node
    node_data: dictionary chứa thông tin đầy đủ của nodes (bao gồm energy)
    r_sen: bán kính truyền tải tối đa (Rsep - subgroup threshold), default: 100m
    R: size threshold - số lượng node tối đa trong 1 cụm, default: 20
    N: số lần thực hiện clustering để chọn kết quả tốt nhất, default: 30
    max_depth: độ sâu đệ quy tối đa
    depth: độ sâu hiện tại
    """
    
    # Điều kiện dừng:
    # 1. Size threshold: len(nodes) <= R
    # 2. Subgroup threshold: max distance <= r_sen
    size_ok = len(nodes) <= R
    distance_ok = check_subgroup_threshold(nodes, r_sen)
    
    if (size_ok and distance_ok) or depth >= max_depth:
        center = np.mean(nodes, axis=0)
        return [{
            "node_ids": node_ids,
            "nodes": nodes,
            "center": center,
            "node_data": node_data if node_data else {}
        }]
    
    # Bước 2: Thực hiện K-means clustering N lần và chọn kết quả tốt nhất
    labels, centers, best_T = kmeans_with_best_T(nodes, N)
    
    # Bước 3: Phân chia thành các subgroups và đệ quy
    clusters = []
    for i in range(2):
        # Lọc nodes và node_ids theo label
        sub_nodes = nodes[labels == i]
        sub_ids = [node_ids[j] for j in range(len(node_ids)) if labels[j] == i]
        
        # Tạo sub_node_data cho cluster con
        sub_node_data = {}
        if node_data:
            for node_id in sub_ids:
                if node_id in node_data:
                    sub_node_data[node_id] = node_data[node_id]
        
        # Đệ quy cho subgroup
        clusters += cluster_split(sub_nodes, sub_ids, sub_node_data, r_sen, R, N, max_depth, depth + 1)
    
    return clusters

In [14]:
def choose_cluster_head(cluster_info, node_data=None):
    """
    Chọn cluster head dựa trên khoảng cách gần nhất với tâm cụm
    Hoặc dựa trên năng lượng cao nhất nếu có node_data
    
    cluster_info: dictionary chứa thông tin cụm
    node_data: dictionary chứa thông tin năng lượng của nodes
    """
    nodes = cluster_info["nodes"]
    center = cluster_info["center"]
    node_ids = cluster_info["node_ids"]
    
    if node_data and len(node_data) > 0:
        # Chọn node có năng lượng cao nhất
        max_energy = -1
        ch_id = node_ids[0]
        for nid in node_ids:
            if nid in node_data and "energy" in node_data[nid]:
                if node_data[nid]["energy"] > max_energy:
                    max_energy = node_data[nid]["energy"]
                    ch_id = nid
        return ch_id
    else:
        # Chọn node gần tâm cụm nhất
        distances = np.linalg.norm(nodes - center, axis=1)
        min_idx = np.argmin(distances)
        return node_ids[min_idx]

# Tạo các thư mục output
input_folder = "input_data"
output_folder = "output_data_kmeans"
os.makedirs(output_folder, exist_ok=True)
draw_folder = "draw_output_kmeans"
os.makedirs(draw_folder, exist_ok=True)

# Xử lý từng file input
for filename in os.listdir(input_folder):
    if filename.startswith("nodes_") and filename.endswith(".json"):
        # Lấy số lượng node từ tên file
        number_nodes = filename.split("_")[1].split(".")[0]
        
        # Đọc dữ liệu từ file input
        with open(os.path.join(input_folder, filename), "r") as f:
            data = json.load(f)
        
        node_positions = np.array([[d["x"], d["y"], d["z"]] for d in data])
        node_ids = [d["id"] for d in data]
        
        # Tạo node_data nếu có thông tin energy
        node_data = {}
        for d in data:
            if "energy" in d:
                node_data[d["id"]] = {"energy": d["energy"]}
        
        # Tính số cụm ước tính
        print(f"File: {filename}")
        print(f"Số node: {len(node_positions)}")
        
        # Phân cụm với thuật toán mới
        # R = size threshold cho mỗi cụm
        # N = số lần thực hiện K-means để chọn kết quả tốt nhất
        R_threshold = max(10, int(number_nodes) // k)  # Đảm bảo R >= 10
        clusters_raw = cluster_split(
            nodes=node_positions, 
            node_ids=node_ids, 
            node_data=node_data if node_data else None,
            R=R_threshold,
            N=30,  # Thực hiện 30 lần để chọn kết quả tốt nhất
            max_depth=10
        )
        
        print(f"Số cụm thực tế: {len(clusters_raw)}")
        
        # Tạo output
        clusters_output = {}
        for i, c in enumerate(clusters_raw):
            ch = choose_cluster_head(c, c["node_data"])
            clusters_output[i] = {
                "nodes": c["node_ids"],
                "center": [float(x) for x in np.round(c["center"], 2)],
                "cluster_head": int(ch),
                "size": len(c["node_ids"])
            }
        
        # Xuất ra file
        out_path = os.path.join(output_folder, f"nodes_{number_nodes}.json")
        with open(out_path, "w") as f:
            json.dump(clusters_output, f, indent=4)
        print(f"Đã xuất file {out_path}")
        
        # Vẽ và lưu hình
        fig = plt.figure(figsize=(10, 8))  # Giảm kích thước để tránh MemoryError
        ax = fig.add_subplot(111, projection='3d')
        
        # Tạo màu sắc cho các cụm - FIX: Sử dụng matplotlib.colormaps thay vì get_cmap
        num_clusters = len(clusters_output)
        if num_clusters <= 10:
            cmap = plt.colormaps.get_cmap('tab10')
        else:
            cmap = plt.colormaps.get_cmap('tab20')
        
        for cid, info in clusters_output.items():
            # Vẽ các node trong cụm
            cluster_nodes = np.array([node_positions[nid] for nid in info['nodes']])
            color = cmap(cid % 20)  # Lấy màu từ colormap
            
            ax.scatter(cluster_nodes[:, 0], cluster_nodes[:, 1], cluster_nodes[:, 2],
                      label=f'Cụm {cid} ({info["size"]})',
                      color=color,
                      alpha=0.6,
                      s=50)
            
            # Vẽ cluster head
            ch_pos = node_positions[info['cluster_head']]
            ax.scatter(ch_pos[0], ch_pos[1], ch_pos[2],
                      color=color,
                      marker='*', 
                      s=300, 
                      edgecolor='black',
                      linewidth=1.5,
                      zorder=100)
            
            # Vẽ tâm cụm
            center = info['center']
            ax.scatter(center[0], center[1], center[2],
                      color=color,
                      marker='x',
                      s=100,
                      linewidth=2,
                      zorder=90)
        
        # Vẽ base station (nếu cần)
        ax.scatter(0, 0, 0, color='red', marker='^', s=400, 
                  label='Base Station', edgecolor='black', linewidth=2, zorder=110)
        
        ax.set_xlabel('X (m)', fontsize=10)
        ax.set_ylabel('Y (m)', fontsize=10)
        ax.set_zlabel('Z (m)', fontsize=10)
        
        # Chỉ hiển thị legend nếu số cụm không quá nhiều
        if num_clusters <= 15:
            ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1), fontsize=8)
        
        plt.title(f'Phân cụm K-Means ({number_nodes} nodes, {num_clusters} cụm)', 
                 fontsize=12, fontweight='bold')
        ax.view_init(elev=25, azim=45)
        
        # FIX: Không dùng bbox_inches='tight' để tránh MemoryError
        # Và giảm DPI xuống 100
        draw_path = os.path.join(draw_folder, f"nodes_{number_nodes}.png")
        try:
            plt.savefig(draw_path, dpi=100)  # Giảm DPI từ 150 xuống 100
            print(f"Đã lưu hình vẽ {draw_path}")
        except MemoryError:
            print(f"MemoryError: Không thể lưu hình với DPI=100, thử DPI=72")
            plt.savefig(draw_path, dpi=72)  # Thử với DPI thấp hơn
            print(f"Đã lưu hình vẽ {draw_path} với DPI=72")
        finally:
            plt.close(fig)
        
        print("-" * 60)

File: nodes_100.json
Số node: 100
Số cụm thực tế: 16
Đã xuất file output_data_kmeans\nodes_100.json
Đã lưu hình vẽ draw_output_kmeans\nodes_100.png
------------------------------------------------------------
File: nodes_1000.json
Số node: 1000
Số cụm thực tế: 40
Đã xuất file output_data_kmeans\nodes_1000.json
Đã lưu hình vẽ draw_output_kmeans\nodes_1000.png
------------------------------------------------------------
File: nodes_20.json
Số node: 20
Số cụm thực tế: 4
Đã xuất file output_data_kmeans\nodes_20.json
Đã lưu hình vẽ draw_output_kmeans\nodes_20.png
------------------------------------------------------------
File: nodes_200.json
Số node: 200
Số cụm thực tế: 35
Đã xuất file output_data_kmeans\nodes_200.json
Đã lưu hình vẽ draw_output_kmeans\nodes_200.png
------------------------------------------------------------
File: nodes_500.json
Số node: 500
Số cụm thực tế: 37
Đã xuất file output_data_kmeans\nodes_500.json
Đã lưu hình vẽ draw_output_kmeans\nodes_500.png
-----------------